<a href="https://colab.research.google.com/github/vinodnbhat/AIML-CEP-2021-Assignments/blob/main/AlexNet_Inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import required libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torch.optim.lr_scheduler as lr_scheduler
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import random
import time

In [2]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

**CIFAR Dataset**

In [3]:
ROOT = '.data'

# Downloading Cifar10 dataset from torchvision datasets
train_data = datasets.CIFAR10(root = ROOT,
                              train = True,
                              download = True)

mean = train_data.data.mean() / 255
std = train_data.data.std() / 255

print(f'Calculated mean: {mean}')
print(f'Calculated std: {std}')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting .data/cifar-10-python.tar.gz to .data
Calculated mean: 0.4733630004850899
Calculated std: 0.2515689250632208


In [4]:
print(train_data.data.shape)

(50000, 32, 32, 3)


In [5]:
train_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean = [mean], std = [std])
                                      ])

test_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean = [mean], std = [std])
                                      ])

In [6]:
train_set = datasets.CIFAR10(root = ROOT,
                             train = True,
                             download = True,
                             transform = train_transforms)

test_set = datasets.CIFAR10(root = ROOT,
                             train = False,
                             download = True,
                             transform = train_transforms)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
print(train_set.data.shape)

(50000, 32, 32, 3)


In [9]:
print(test_set.data.shape)

(10000, 32, 32, 3)


In [10]:
print(f'Number of training samples: {len(train_set)}')
print(f'Number of testing samples: {len(test_set)}')

Number of training samples: 50000
Number of testing samples: 10000


In [11]:
batch_size = 64

train_loader = data.DataLoader(train_set,
                               shuffle = True,
                               batch_size = batch_size)

test_loader = data.DataLoader(test_set,
                              batch_size = batch_size)

In [14]:
# Checking the bacth dimensions
for images, labels in train_loader:
  print('Image batch dimensions:', images.shape)
  print('Label batch dimensions:', labels.shape)
  break

Image batch dimensions: torch.Size([64, 3, 32, 32])
Label batch dimensions: torch.Size([64])


In [15]:
### Model Settings ###

# Hyperparameters
learning_rate = 0.01

# Architecture
num_classes = 10

In [56]:
class AlexNet(nn.Module):
  def __init__(self, num_classes = 10):
    super(AlexNet, self).__init__()
    self.features = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size = 3, stride = 2, padding = 1),
        nn.ReLU(inplace = True),
        nn.MaxPool2d(kernel_size = 2),
        nn.Conv2d(64, 192, kernel_size = 3, padding = 1),
        nn.ReLU(inplace = True),
        nn.MaxPool2d(kernel_size = 2),
        nn.Conv2d(192, 384, kernel_size = 3, padding = 1),
        nn.ReLU(inplace = True),
        nn.Conv2d(384, 256, kernel_size = 3, padding = 1),
        nn.ReLU(inplace = True),
        nn.Conv2d(256, 256, kernel_size = 3, padding = 1),
        nn.ReLU(inplace = True),
        nn.MaxPool2d(kernel_size = 2)
    )
    self.classifier = nn.Sequential(
        nn.Dropout(),
        nn.Linear(256 * 2 * 2, 4096),
        nn.ReLU(inplace = True),
        nn.Dropout(),
        nn.Linear(4096, 4096),
        nn.ReLU(inplace = True),
        nn.Linear(4096, num_classes)
    )

  def forward(self, x):
    x = self.features(x)
    x = x.view(x.size(0), 256 * 2 * 2)
    x = self.classifier(x)
    logits = x
    probas = F.log_softmax(x, dim = 1)
    return logits, probas

In [57]:
# Initialize the model
model = AlexNet()

In [58]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 23,272,266 trainable parameters


In [59]:
optimizer = torch.optim.Adam(model.parameters(), lr =learning_rate)

In [60]:
lossfn = nn.CrossEntropyLoss()

In [61]:
if torch.cuda.is_available():
  print('cuda is available. Using cuda..')
else:
  print('cuda is not available. Using CPU..')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

cuda is not available. Using CPU..


In [62]:
model = model.to(device)
lossfn = lossfn.to(device)

In [63]:
def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time / 60)
  elapsed_secs = elapsed_time - (elapsed_mins * 60)
  return elapsed_mins, elapsed_secs

In [64]:
# Compute the Accuracy
def compute_accuracy(model, data_loader):
  correct_pred, num_examples = 0, 0
  for i , (features, targets) in enumerate(data_loader):
    features = features.to(device)
    targets = targets.to(device)
    logits, probas = model(features)
    _, predicted_labels = torch.max(probas, 1)
    num_examples += targets.size(0)
    correct_pred += (predicted_labels == targets).sum()
  return correct_pred.float() / num_examples *100

In [67]:
def train(model, iterator, optimizer, criterion, device):
  epoch_loss = 0

  model.train()

  for (x, y) in iterator:

    x = x.to(device)
    y = y.to(device)

    optimizer.zero_grad()

    y_pred_logits, y_pred_probas = model(x)

    loss = lossfn(y_pred_logits, y)

    loss.backward()

    optimizer.step()

    epoch_loss += loss.item()

  return epoch_loss / len(iterator)

In [68]:
save_model = False
patience_early_stopping = 3 #training will stop if model performance does not improve for these many consecutive epochs
cnt = 0 #counter for checking patience level
EPOCHS = 3
prev_test_acc = 0 #initializing prev test accuracy for early stopping condition
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'max', factor = 0.2, patience = 1) #learning rate scheduler, update learning rate by 
#factor of 0.2 if test accuracy does not improve for patience+1 consecutive epochs

for epoch in range(EPOCHS):
  print("current learning rate", optimizer.state_dict()['param_groups'][0]['lr'])
  start_time = time.perf_counter()

  train_loss = train(model, train_loader, optimizer, lossfn, device)
  train_acc = compute_accuracy(model, train_loader)

  if save_model:
    torch.save(model.state.dict(), 'alexnet_model.pt')

  if epoch % 1 == 0: #for every epoch we shall compute the test accuracy
    test_acc = compute_accuracy(model, test_loader)

    if test_acc > prev_test_acc: #check if test accuracy for current epoch has improved compared to previous epoch
      cnt = 0 #if accuracy improves reset counter to 0
    else:
      cnt += 1 #otherwise increment current counter

    prev_test_acc = test_acc

  scheduler.step(test_acc) #updates learning rate

  end_time = time.perf_counter()

  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  print(f'Epoch: {epoch+1:2} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc:.2f}%')
  if epoch % 1 == 0:  #for every epoch we shall print the test loss and test accuracy 
    print(f'\tTest Acc: {test_acc:.2f}% \n')

  if cnt == patience_early_stopping:
    print(f'early stopping as accuracy did not improve for {patience_early_stopping} consecutive epochs')
    break



current learning rate 0.01
Epoch:  1 | Epoch Time: 13m 34.10375407799984s
	Train Loss: 52.262 | Train Acc: 10.05%
	Test Acc: 9.89% 

current learning rate 0.01
Epoch:  2 | Epoch Time: 16m 59.50214466899888s
	Train Loss: 2.304 | Train Acc: 10.00%
	Test Acc: 10.00% 

current learning rate 0.01
Epoch:  3 | Epoch Time: 16m 40.9703032489997s
	Train Loss: 2.304 | Train Acc: 10.00%
	Test Acc: 10.00% 

